In [11]:
import numpy as np
import os

os.environ['DEBUG'] = '1'

from tinygrad.helpers import Timing
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD

from lib.utils import get_mnist

In [12]:
X_train, Y_train, X_test, Y_test = get_mnist("../data")

In [13]:
class TinyNet:
    def __init__(self):
        self.l1 = Linear(784, 128, bias=False)
        self.l2 = Linear(128, 10, bias=False)

    def __call__(self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        return x

net = TinyNet()

In [14]:
opt = SGD([net.l1.weight, net.l2.weight], lr=3e-4)

In [16]:
with Tensor.train():
    for step in range(1000):
        # random sample a batch
        samp = np.random.randint(0, X_train.shape[0], size=(64))
        batch = Tensor(X_train[samp], requires_grad=False)
        # get the corresponding labels
        labels = Tensor(Y_train[samp])

        # forward pass
        out = net(batch)
        # compute loss
        loss = out.sparse_categorical_crossentropy(labels)

        # zero gradients
        opt.zero_grad()

        # backward pass
        loss.backward()

        # update parameters
        opt.step()

        # calculate accuracy
        pred = out.argmax(axis=-1)
        acc = (pred == labels).mean()

        if step % 100 == 0:
            print(f"Step {step+1} | Loss: {loss.numpy():0.4f} | Accuracy: {acc.numpy():0.4f}")    

Step 1 | Loss: 2.2292 | Accuracy: 0.2969
Step 101 | Loss: 2.2327 | Accuracy: 0.2812
Step 201 | Loss: 2.2316 | Accuracy: 0.3125
Step 301 | Loss: 2.2431 | Accuracy: 0.2344
Step 401 | Loss: 2.1986 | Accuracy: 0.3906
Step 501 | Loss: 2.2214 | Accuracy: 0.3125
Step 601 | Loss: 2.2249 | Accuracy: 0.3125
Step 701 | Loss: 2.2249 | Accuracy: 0.2812
Step 801 | Loss: 2.2247 | Accuracy: 0.2812
Step 901 | Loss: 2.1997 | Accuracy: 0.3281


In [18]:
with Timing("Time: "):
    avg_acc = 0
    for step in range(1000):
        # random sample a batch
        samp = np.random.randint(0, X_test.shape[0], size=(64))
        batch = Tensor(X_test[samp], requires_grad=False)
        # get the corresponding labels
        labels = Y_test[samp]

        # forward pass
        out = net(batch)

        # calculate accuracy
        pred = out.argmax(axis=-1).numpy()
        avg_acc += (pred == labels).mean()
    print(f"Test Accuracy: {avg_acc / 1000:0.4f}")

Test Accuracy: 0.3912
Time: 6174.39 ms
